In [1]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', 'LM(중국어)', 'LM(영어)', '번역(중-한)', '번역(한-중)', '한국어 번역 교정', '번역(영-한)', '번역(한-영)']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            domain = f'{line['domain']} {line['subdomain']}'
            source_language = line['source_language']
            source_text = line[source_language]
            target_language = line['target_language']
            target_text_mt = line['mt']
            target_text = line[target_language]
            text_kr = None
            if 'ko' in line:
                text_kr = line['ko']
            text_cn = None
            if 'cn' in line:
                text_cn = line['cn']
            text_en = None
            if 'en' in line:
                text_en = line['en']
            #### data preprocess end 
            
            if text_kr:
                #### LM(한국어)
                data = {'text': None}
                ## preprocess data from line
                data['text'] = text_kr
                ## preprocess data from line end
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### LM(한국어) end
            
            if text_cn:
                #### LM(중국어)
                data = {'text': None}
                ## preprocess data from line
                data['text'] = text_cn
                ## preprocess data from line end
                task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### LM(중국어) end
            
            if text_en:
                #### LM(영어)
                data = {'text': None}
                ## preprocess data from line
                data['text'] = text_en
                ## preprocess data from line end
                task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### LM(영어) end
            
            if text_cn and text_kr:
                #### 번역(중-한)
                data = {'input': None, 'output': None}
                ## preprocess data from line
                data['input'] = text_cn
                data['output'] = text_kr
                ## preprocess data from line end
                task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 번역(중-한) end
                
                
                #### 번역(한-중)
                data = {'input': None, 'output': None}
                ## preprocess data from line
                data['input'] = text_kr
                data['output'] = text_cn
                ## preprocess data from line end
                task_files[4].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 번역(한-중) end
            
            if target_text != target_text_mt:
                #### 한국어 번역 교정
                ### 'output' is list of output sorted by relevance (1st is most relevant)
                data = {'input': None, 'output': None}
                ## preprocess data from line
                data['input'] = source_text
                data['output'] = [target_text, target_text_mt]
                ## preprocess data from line end
                task_files[5].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 한국어 번역 교정 end
            
            if text_en and text_kr:
                #### 번역(영-한)
                data = {'input': None, 'output': None}
                ## preprocess data from line
                data['input'] = text_en
                data['output'] = text_kr
                ## preprocess data from line end
                task_files[6].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 번역(영-한) end
                
                
                #### 번역(한-영)
                data = {'input': None, 'output': None}
                ## preprocess data from line
                data['input'] = text_kr
                data['output'] = text_en
                ## preprocess data from line end
                task_files[7].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 번역(한-영) end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:07<00:00,  7.44s/it]
